# pYPK0_TDH3_SsXYL2_PGI

This notebook describes the assembly of the [_Saccaromyces cerevisiae_](www.yeastgenome.org)
single gene expression vector pYPK0_TDH3_SsXYL2_PGI.

It is made by _in-vivo_ homologous recombination between three PCR products and one linear vector fragment.
The PCR products are a promoter generated from a pYPK_Z vector, a gene from a template and 
a terminator from a pYPKa_E vector. The three PCR products are joined to
a linearized [pYPKpw](https://github.com/BjornFJohansson/ypk-xylose-pathways/blob/master/pYPKpw.gb) 
backbone vector that has the [URA3](http://www.yeastgenome.org/locus/S000000747/overview) 
marker and a _S. crevisiae_ [2 micron](http://blog.addgene.org/plasmids-101-yeast-vectors) origin of replication. 

The four linear DNA fragments are joined by homologous recombination in a 
[_Saccharomyces cerevisiae_](http://wiki.yeastgenome.org/index.php/Commonly_used_strains) ura3 mutant.

![pYPK0_promoter_gene_terminator](tp_g_tp.png "pYPK0_promoter_gene_terminator")

The [pydna](https://pypi.python.org/pypi/pydna/) package is imported in the code cell below. 
There is a [publication](http://www.biomedcentral.com/1471-2105/16/142) describing pydna as well as
[documentation](http://pydna.readthedocs.org/en/latest/) available online. 
Pydna is developed on [Github](https://github.com/BjornFJohansson/pydna).

In [1]:
from pydna.parsers import parse_primers
from pydna.readers import read
from pydna.design import primer_design
from pydna.amplify import pcr
from pydna.assembly import Assembly

The YPK [standard primers](standard_primers.txt) are read into a dictionary in the code cell below.

In [2]:
p = { x.id: x for x in parse_primers("standard_primers.txt") }

The backbone vector [pYPKpw](pYPKpw.gb) is read from a local file in the code cell below.

In [3]:
pYPKpw = read("pYPKpw.gb")

The backbone vector is linearized by digestion with [EcoRV](http://rebase.neb.com/rebase/enz/EcoRV.html).
The restriction enzyme functionality is provided by [biopython](http://biopython.org).

In [4]:
from Bio.Restriction import EcoRV

pYPK_EcoRV = pYPKpw.linearize(EcoRV)

The pYPKa derived _E. coli_ plasmids containing the [promoter](pYPKa_Z_TDH3.gb) and [terminator](pYPKa_E_PGI.gb) 
as well as the [gene template](SsXYL2.gb) sequence are read into three variables in the code cell below.

In [5]:
promoter_template   = read("pYPKa_Z_TDH3.gb")
gene_template       = read("SsXYL2.gb")
terminator_template = read("pYPKa_E_PGI.gb")

The construction of the two vector above are described in the [pYPKa_ZE_TDH3](pYPKa_ZE_TDH3.ipynb) notebooks.

The promoter is amplified with from [pYPKa_Z_TDH3](pYPKa_Z_TDH3.gb). A suggested PCR program can be found at the end of this document.

In [6]:
prom = pcr( p['577'], p['567'], promoter_template)

Primers with tails are needed for the recombination between the gene and the promoter/terminator PCR products.
The tails are designed to provide 33 bp of terminal homology between the DNA fragments.

In [7]:
fp_tail = "tgcccactttctcactagtgacctgcagccgacAA"
rp_tail = "AAatcctgatgcgtttgtctgcacagatggCAC"

Primers with the tails above are designed for the gene template in the code cell below.

In [8]:
ins = primer_design(gene_template)
fp = fp_tail + ins.forward_primer
rp = rp_tail + ins.reverse_primer

The primers are included in the [new_primers.txt](new_primers.txt) list and in the end of the [pathway notebook](pw.ipynb) file.

In [9]:
print(fp.format("fasta"))
print(rp.format("fasta"))
with open("new_primers.txt", "a+") as f:
    f.write(fp.format("fasta"))
    f.write(rp.format("fasta"))

>fw1092 SsXYL2
tgcccactttctcactagtgacctgcagccgacAAATGACTGCTAACCCTTCC

>rv1092 SsXYL2
AAatcctgatgcgtttgtctgcacagatggCACTTACTCAGGGCCGTCA



The gene is amplifed using the newly designed primers. A suggested PCR program can be found at the end of this document.

In [10]:
gene = pcr( fp, rp, gene_template)

The terminator is amplified from [pYPKa_E_PGI](pYPKa_E_PGI.gb). A suggested PCR program can be found at the end of this document.

In [11]:
term = pcr( p['568'], p['578'], terminator_template)

The four linear DNA fragments are mixed and transformed
to a _Saccharomyces cerevisiae_ ura3 mutant.

The fragments will be assembled by _in-vivo_ [homologous recombination](http://www.ncbi.nlm.nih.gov/pubmed/2828185):

In [12]:
asm = Assembly( (pYPK_EcoRV, prom, gene, term), limit=31 )

asm

Assembly:
Sequences........................: [5603] [929] [1160] [1339]
Sequences with shared homologies.: [5603] [929] [1339] [1160]
Homology limit (bp)..............: 31
Number of overlaps...............: 4
Nodes in graph(incl. 5' & 3')....: 6
Only terminal overlaps...........: No
Circular products................: [8580]
Linear products..................: [8824] [8721] [8613] [8613] [7825] [7518] [7486] [6698] [6391] [3362] [2466] [2056] [244] [141] [33] [33]

The representation of the asm object above should normally indicate one circcular product only.  
More than one circular products might indicate an incorrect assembly strategy or represent
by-products that might arise in the assembly process.  
The largest recombination product is chosen as candidate for the pYPK0_TDH3_SsXYL2_PGI vector.

In [13]:
candidate = asm.circular_products[0]

candidate.figure()

 -|pYPKpw|124
|         \/
|         /\
|         124|929bp_PCR_prod|33
|                            \/
|                            /\
|                            33|1160bp_PCR_prod|33
|                                               \/
|                                               /\
|                                               33|1339bp_PCR_prod|242
|                                                                  \/
|                                                                  /\
|                                                                  242-
|                                                                     |
 ---------------------------------------------------------------------

The candidate vector is synchronized to the backbone vector. This means that
the plasmid origin is shifted so that it matches the pYPKpw backbone vector.

In [14]:
result = candidate.synced(pYPKpw)

### Diagnostic PCR confirmation

The structure of the final vector is confirmed by two
separate PCR reactions, one for the promoter and gene and
one for the gene and terminator.

PCR using standard primers 577 and 467 to amplify promoter and gene.

In [15]:
product = pcr( p['577'], p['467'], result)

A correct clone should give this size in base pairs:

In [16]:
print(len(product))

2060


If the promoter is missing from the assembly, the PCR product will have this size in base pairs:

In [17]:
print(len(product) - len(prom))

1131


If the gene is missing from the assembly, the PCR product will have this size in base pairs:

In [18]:
print(len(product) - len(gene))

900


PCR using standard primers 468 and 578 to amplify gene and terminator.

In [19]:
product2 = pcr( p['468'], p['578'], result)

A correct clone should give this size:

In [20]:
print(len(product2))

2483


If the gene is missing from the assembly, the PCR product will have this size in base pairs:

In [21]:
print(len(product2) - len(gene))

1323


If the terminator is missing from the assembly, the PCR product will have this size in base pairs:

In [22]:
print(len(product2) - len(term))

1144


The cseguid checksum for the resulting plasmid is calculated for future reference.
The [cseguid checksum](http://pydna.readthedocs.org/en/latest/pydna.html#pydna.utils.cseguid) 
uniquely identifies a circular double stranded sequence.

In [23]:
result.cseguid()

UdTwzMX6S8kBTgDnzL1y_glnGCM

The file is named based on the names of promoter, gene and terminator.

In [24]:
result.locus = "pYPK0_tp_g_tp"
result.definition = "pYPK0_TDH3_SsXYL2_PGI"

Sequence is stamped with cseguid checksum. This can be used to verify the 
integrity of the sequence file.

In [25]:
result.stamp()

cSEGUID_UdTwzMX6S8kBTgDnzL1y_glnGCM

Write sequence to a local file.

In [26]:
result.write("pYPK0_TDH3_SsXYL2_PGI.gb")

pYPK0_TDH3_SsXYL2_PGI.gb

## PCR programs for the amplification of Promoter, Gene and Terminator

Promoter

In [27]:
prom.program()


Taq (rate 30 nt/s) 35 cycles             |929bp
95.0°C    |95.0°C                 |      |Tm formula: Biopython Tm_NN
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|Primer1C 1.0µM
|         |      \ 55.8°C/ 0min28s| 5min |Primer2C 1.0µM
|         |       \_____/         |      |GC 39%
|         |         30s           |      |4-12°C

Gene

In [28]:
gene.program()


Taq (rate 30 nt/s) 35 cycles             |1160bp
95.0°C    |95.0°C                 |      |Tm formula: Biopython Tm_NN
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|Primer1C 1.0µM
|         |      \ 57.2°C/ 0min35s| 5min |Primer2C 1.0µM
|         |       \_____/         |      |GC 49%
|         |         30s           |      |4-12°C

Terminator

In [29]:
term.program()


Taq (rate 30 nt/s) 35 cycles             |1339bp
95.0°C    |95.0°C                 |      |Tm formula: Biopython Tm_NN
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|Primer1C 1.0µM
|         |      \ 56.3°C/ 0min40s| 5min |Primer2C 1.0µM
|         |       \_____/         |      |GC 39%
|         |         30s           |      |4-12°C

## Download [pYPK0_TDH3_SsXYL2_PGI](pYPK0_TDH3_SsXYL2_PGI.gb)

In [30]:
import pydna
reloaded = read("pYPK0_TDH3_SsXYL2_PGI.gb")
reloaded.verify_stamp()


cSEGUID_UdTwzMX6S8kBTgDnzL1y_glnGCM